In [1]:
%config Completer.use_jedi = False

In [1]:
import os
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import matplotlib.ticker as mticker
import mne
import json

# import scipy.io as sio
# from scipy import signal

import pac

In [2]:
def plot_pac(pac, high_freq, low_freq):
    fig = plt.figure(figsize=(7, 15))
    ax = fig.subplots()
    im = ax.imshow(pac, origin='lower', interpolation='nearest')

    xticks_num = (low_freq[1] - low_freq[0]) / 5
    yticks_num = (high_freq[1] - high_freq[0]) / 10

    ax.xaxis.set_major_locator(mticker.MaxNLocator(5))
    ticks_loc = ax.get_xticks()
    ax.xaxis.set_major_locator(mticker.FixedLocator(ticks_loc))
    xticks = [''] + [int(n) for n in np.linspace(low_freq[0],
                                                 low_freq[1], ticks_loc.shape[0]-2).tolist()] + ['']
    ax.set_xticklabels(xticks)

    ax.yaxis.set_major_locator(mticker.MaxNLocator(10))
    ticks_loc = ax.get_yticks()
    ax.yaxis.set_major_locator(mticker.FixedLocator(ticks_loc))
    yticks = [''] + [int(n) for n in np.linspace(high_freq[0],
                                                 high_freq[1], ticks_loc.shape[0]-2).tolist()] + ['']
    ax.set_yticklabels(yticks)

    plt.show()


def create_elc_file(task):
    with open(os.path.join(task.dir, task.file_formatter.format('electrodes.elc')), 'w') as elc:
        locs = pd.read_csv(os.path.join(
            task.dir, task.file_formatter.format('electrodes.tsv')), sep="\t")
        locs = locs.iloc[:64]
        locs = locs.append(pd.DataFrame([{'name': 'RPA', 'x': 0, 'y': 1, 'z': 0},
                                  {'name': 'LPA', 'x': 0, 'y': -1, 'z': 0},
                                  {'name': 'Nz', 'x': 1, 'y': 0, 'z': 0}, ]))
        elc.write('\n'.join([
            '# ASA electrode file',
            'ReferenceLabel	avg',
            'UnitPosition	mm',
            f'NumberPositions=	{locs.shape[0]}',
            'Positions',
        ]))
        elc.write('\n')
        elc.write(
            '\n'.join(locs.agg(lambda loc: f'{loc.x}\t{loc.y}\t{loc.z}', axis=1)))
        elc.write('\n')
        elc.write('Labels\n')
        elc.write('\n'.join(locs.name))

In [3]:

def create_tasks_df(df):
    tasks = []

    for _, participant in df.iterrows():
        if participant.Group == 'PD':
            sessions = [(1, 1 * (participant.sess1_Med == 'ON')),
                        (2, 1 * (participant.sess2_Med == 'ON'))]
        else:
            sessions = [(1, 2)]

        for sess, pd_drug_type in sessions:
            participant_tasks = {}
            participant_tasks['participant_id'] = participant['participant_id']
            participant_tasks['pd_drug_type'] = pd_drug_type
            participant_tasks['isMale'] = participant['sex'] == 'Male'
            participant_tasks['age'] = participant['age']
            participant_tasks['dir'] = os.path.join(DS_PATH, participant['participant_id'], f'ses-{sess:02}', 'eeg',)
            participant_tasks['file'] = f'{participant["participant_id"]}_ses-{sess:02}_eeg_{participant["participant_id"]}_ses-{sess:02}_task-Rest_eeg.mat'
            participant_tasks['file_formatter'] = f'{participant["participant_id"]}_ses-{sess:02}_task-Rest_{{}}'
            participant_tasks['path'] = os.path.join(
                participant_tasks['dir'], participant_tasks['file'])
            participant_tasks['pre'] = os.path.join(
                participant_tasks['dir'], 
                
                f'{participant["participant_id"]}_ses-{sess:02}_eeg_{participant["participant_id"]}_ses-{sess:02}_task-Rest_eeg.mat'
            )

            tasks.append(participant_tasks)

    tasks_df = pd.DataFrame(tasks)

    return tasks_df


def _test_tasks_df(tasks_df, i=0):
    task = tasks_df.iloc[i]

    assert os.path.exists(task.path)

    ds = sio.loadmat(task.path)

    ds['data'] = ds['EEG']

    nbchan = ds['data'][0, 0]['nbchan'][0, 0]  # .dtype
    Fs = ds['data'][0, 0]['srate'][0, 0]
    times = ds['data'][0, 0]['times']
    data = ds['data'][0, 0]['data']

    dtypes = [k for k in ds['data'][0, 0]['event'].dtype.names]
    events = pd.DataFrame([{n: event[n].item() if event[n].size > 0 else None for n in dtypes}
                           for event in ds['data'][0, 0]['event'][0]])

    electrodes = pd.read_csv(os.path.join(
        task['dir'], task['file_formatter'].format('electrodes.tsv')), sep='\t')

    reject = ds['data'][0, 0]['reject']

    print(events)


# Create Task list in `tasks_df`

In [4]:
def check_completed(task, event=None) -> bool:
    json_path = os.path.join(task['dir'], task['file_formatter'].format('completed_1ch.json'))
    completed = {}
    if os.path.exists(json_path):
        with open(json_path) as f:
            completed = json.load(f)
    
    if event is None:
        return completed.get('total', False)
    else:
        return completed.get(event, False)


def update_completed(task, event=None) -> bool:
    json_path = os.path.join(task['dir'], task['file_formatter'].format('completed_1ch.json'))
    completed = {}
    if os.path.exists(json_path):
        with open(json_path) as f:
            completed = json.load(f)

    if event is None:
        completed['total'] = True
    else:
        completed[event] = True
        
    with open(json_path, 'w') as f:
        json.dump(completed, f)

In [5]:
def analyse_erps(erps: dict, task=None):
    mvls = {}
    mvl_2ds = {}

    for event_type, erp in erps.items():
        mvl_2d = np.zeros(
            (erp.info['nchan'], erp.info['nchan'], 200-32+1, 40-4+1))
        mvl = np.zeros((erp.info['nchan'], erp.info['nchan'], ))
        tfds = {}

        erp_df = erp.to_data_frame()
        erp_df = erp_df.set_index('time')

        if task is not None:
            print(f'{task.participant_id} {event_type} tfd started')
        
        for ch in erp_df:
            tfd = pac.rid_rihaczek(erp_df[ch], int(erp.info['sfreq']))
            tfds[ch] = tfd

        for chx, chxname in enumerate(erp_df):
            chy = chx
            chyname = chxname
            # for chy, chyname in enumerate(erp_df):
            # print(chxname, chyname)

            # todo:
            # if(check_completed(task, f'{event_type}_{chxname}_{chyname}')):
            #     continue

            mvl_2d[chx, chy] = pac.tfMVL_tfd2_2d(
                tfds[chxname], tfds[chyname], [32, 200], [4, 40])
            mvl[chx, chy] = pac.tfMVL_tfd2(
                tfds[chxname], tfds[chyname], [32, 200], [4, 40])
            
        mvls[event_type] = mvl
        mvl_2ds[event_type] = mvl_2d

    return mvls, mvl_2ds


In [6]:
def analyse_sub(task):
    if(check_completed(task)):
        return

    raw = mne.io.read_raw_eeglab(os.path.join(task['dir'], 'pre_' + task['file_formatter'].format('eeg.set')),
                                 preload=True, verbose=0)
    # raw.drop_channels(['X', 'Y', 'Z'])
    raw.set_eeg_reference()
    
    for ch in raw._data:
        ch /= ch.std()

    create_elc_file(task)
    montage = mne.channels.read_custom_montage(os.path.join(
        task.dir, task.file_formatter.format('electrodes.elc')))
    raw.set_montage(montage)

    eeg_picks = mne.pick_types(raw.info, eeg=True, meg=False, eog=True)
    freqs = (60, 120, 180, 240)
    raw_notch = raw.copy().notch_filter(freqs=freqs, picks=eeg_picks, verbose=0)
    raw_filtered = raw_notch.copy().filter(l_freq=1, h_freq=150, verbose=0)

    events, event_dict = mne.events_from_annotations(raw_filtered, verbose=0)
    epochs = mne.Epochs(raw_filtered, events, event_id=event_dict,
                        tmin=-0.2, tmax=1, preload=True, verbose=0)

    selected_events = ['S200', 'S201', 'S202']
    erps = {}
    for ev in selected_events:
        erps[ev] = epochs[ev].average()
        
    mvls, mvl_2ds = analyse_erps(erps, task)
    np.savez_compressed(os.path.join(task['dir'], task['file_formatter'].format('mvls')),
                        **mvls)
    np.savez_compressed(os.path.join(task['dir'], task['file_formatter'].format('mvl_2ds')),
                        **mvl_2ds)
    update_completed(task)

In [8]:
if __name__ == '__main__':
    # BASE_PATH = os.path.join(os.getenv('HOME'), 'DS') 
    DS_PATH = os.path.join(os.getenv('HOME'), 'DS', 'ds003490-download')
    # BASE_PATH = 'G:\\filmuniversity\\Master sharif\\MasterProject\\data'
    # DS_PATH = 'G:\\filmuniversity\\Master sharif\\MasterProject\\data\\parkinsons-oddball'

    df = pd.read_csv(os.path.join(DS_PATH, 'participants.tsv'), sep="\t")

    tasks_df = create_tasks_df(df)

    # __test__ = 1
    if '__test__' in locals():
        _test_tasks_df(tasks_df, 0)

    # analyse_sub(tasks_df.iloc[0])

    # from multiprocessing import Pool
    # with Pool(4) as p:
    #    p.map(analyse_sub, tasks_df.iloc)

    # for task in tasks_df.iloc:
    #     analyse_sub(task)
    print('finished')




finished


In [21]:
task = tasks_df.iloc[0]
if(check_completed(task)):
    print('have completed')

raw = mne.io.read_raw_eeglab(os.path.join(task['dir'], 'pre_' + task['file_formatter'].format('eeg.set')),
                             preload=True, verbose=0)
# raw.drop_channels(['X', 'Y', 'Z'])
raw.set_eeg_reference()

have completed


<ipython-input-21-5bb6fe260f1f>:6: RuntimeWarning: The data contains 'boundary' events, indicating data discontinuities. Be cautious of filtering and epoching around these events.
  preload=True, verbose=0)


<RawEEGLAB | pre_sub-001_ses-01_task-Rest_eeg.fdt, 64 x 294100 (588.2 s), ~143.7 MB, data loaded>

In [22]:
raw._data

array([[ 9.11737917e-06,  7.58845948e-06,  3.16737128e-06, ...,
         6.15724038e-06,  3.73623968e-06,  7.81553205e-07],
       [-4.26202137e-07, -9.99983089e-07, -4.82658794e-08, ...,
         1.36039781e-06, -1.03442219e-07, -9.68883340e-07],
       [ 2.70234612e-08,  1.50932334e-06,  2.57471514e-06, ...,
         3.16473483e-06,  2.37565900e-06,  6.68393489e-07],
       ...,
       [ 2.29394553e-06,  3.54992165e-07, -3.83743518e-07, ...,
        -4.31960393e-06, -3.35159849e-06, -4.00139952e-06],
       [ 5.59396790e-06,  1.90658449e-06, -4.78750038e-06, ...,
        -1.08043060e-05, -7.02668141e-06, -6.21006155e-06],
       [ 6.17957590e-06,  4.72613905e-06, -1.66502833e-06, ...,
        -2.37263203e-06, -1.71977996e-06, -1.98304236e-06]])

In [23]:
for ch in raw._data:
    ch /= ch.std()

In [24]:
raw.get_data()

array([[ 1.74037898,  1.44852979,  0.60460647, ...,  1.17533027,
         0.71319541,  0.14918747],
       [-0.15196008, -0.35653859, -0.01720894, ...,  0.48504252,
        -0.03688177, -0.34545015],
       [ 0.00764866,  0.42719541,  0.72874146, ...,  0.89573927,
         0.67240106,  0.18918056],
       ...,
       [ 0.44605894,  0.06902842, -0.07461913, ..., -0.8399493 ,
        -0.65172013, -0.77807428],
       [ 0.87509736,  0.29825824, -0.74893689, ..., -1.69018125,
        -1.09922517, -0.97147652],
       [ 1.54872602,  1.18446551, -0.41728959, ..., -0.59462931,
        -0.43101145, -0.4969903 ]])